In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm


In [2]:
df = pd.read_csv(rf"C:\Users\lihou\Box\BudgetAnnouncements\data\step_3.csv")

In [3]:
df

,date,year,dow,note,date_n5,date_p1,icr_d,icrf_d,ivwir_d,ilr_sq_d,...,ois_12y_d,ois_15y_d,ois_20y_d,ois_25y_d,ois_30y_d,ois_40y_d,ois_50y_d,article_count,q1_ratio,q2_ratio
0,1980-03-26,1980,wed,spring_1,1980-03-19,1980-03-27,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,140,-0.589744,-0.661017
1,1981-03-10,1981,tue,spring_1,1981-03-03,1981-03-11,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,236,-0.369565,-0.776224
2,1982-03-09,1982,tue,spring_1,1982-03-02,1982-03-10,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,258,-0.375000,-0.671053
3,1982-11-08,1982,mon,autumn_1,1982-11-01,1982-11-09,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68,-0.800000,-0.777778
4,1983-03-15,1983,tue,spring_1,1983-03-08,1983-03-16,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,210,-0.616438,-0.698113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,2022-03-23,2022,wed,spring_1,2022-03-16,2022-03-24,0.137656,-1.041333,-1.017542,-0.180219,...,0.0281,0.0365,0.04700,0.04780,0.04900,0.0457,0.04140,26,0.500000,-0.882353
87,2022-09-23,2022,fri,mini_1,2022-09-15,2022-09-26,-1.611567,-0.991346,-0.852320,1.443618,...,1.0204,1.0015,0.96050,0.94050,0.92650,0.8987,0.87720,67,0.658537,-0.846154
88,2022-11-17,2022,thu,autumn_1,2022-11-10,2022-11-18,0.040904,-1.164408,-0.992952,-0.136375,...,-0.0508,-0.0580,-0.06190,-0.06080,-0.05980,-0.0656,-0.06910,33,0.058824,-1.000000
89,2023-03-15,2023,wed,spring_1,2023-03-08,2023-03-16,-1.597111,0.642808,0.655452,1.059468,...,-0.2450,-0.2400,-0.23000,-0.22100,-0.21200,-0.1974,-0.18930,24,-0.777778,-0.875000


In [4]:
def regression(df, y_col, x_col, hkm_col=None):
    if hkm_col:
        df_temp = df[["year", "article_count", x_col, y_col, hkm_col]]
    else:
        df_temp = df[["year", "article_count", x_col, y_col]]

    na_info = df_temp[df_temp.isna().any(axis=1)]

    rows_dropped = []

    if not na_info.empty:
        for _, row in na_info.iterrows():
            na_columns = row[row.isna()].index.tolist()
            rows_dropped.append(
                f"Year {row['year']}, Date {row.index}: Column {na_columns}"
            )

    df_temp = df_temp.dropna()

    if len(df_temp) == 0:
        return "no data"
    if len(df_temp) < 5:
        return "too few data points"

    if hkm_col:
        df_temp["interaction"] = df_temp[x_col] * df_temp[hkm_col]
        X = df_temp[[x_col, hkm_col, "interaction"]]
    else:
        X = df_temp[x_col]

    y = df_temp[y_col]
    X = sm.add_constant(X)

    try:
        model = sm.OLS(y, X).fit()

        params = model.params
        r_squared = model.rsquared
        t_values = model.tvalues

        print("Regression:")
        if not hkm_col:
            print(f"{y_col} ~ {x_col}")
        else:
            print(f"{y_col} ~ {x_col} + {hkm_col} + interaction")
        print(f"Parameters:")
        print(params)
        print(f"R2: {r_squared}")
        print(f"T-values:")
        print(t_values)
        print()
        return model

    except ValueError:
        print(f"ValueError: {y_col} ~ {x_col}")
        return "value error"


In [5]:
df.columns.tolist()

['date',
 'year',
 'dow',
 'note',
 'date_n5',
 'date_p1',
 'icr_d',
 'icrf_d',
 'ivwir_d',
 'ilr_sq_d',
 '2.5_n_d',
 '5.0_n_d',
 '10.0_n_d',
 '15.0_n_d',
 '20.0_n_d',
 '30.0_n_d',
 '2.5_r_d',
 '5.0_r_d',
 '10.0_r_d',
 '15.0_r_d',
 '20.0_r_d',
 '30.0_r_d',
 '2.5_i_d',
 '5.0_i_d',
 '10.0_i_d',
 '15.0_i_d',
 '20.0_i_d',
 '30.0_i_d',
 'gbpusd_o_d',
 'gbpusd_c_d',
 'ftse_o_d',
 'ftse_c_d',
 'gbpeur_o_d',
 'gbpeur_c_d',
 'rpi_1m_d',
 'rpi_3m_d',
 'rpi_6m_d',
 'rpi_9m_d',
 'rpi_1y_d',
 'rpi_2y_d',
 'rpi_3y_d',
 'rpi_4y_d',
 'rpi_5y_d',
 'rpi_6y_d',
 'rpi_7y_d',
 'rpi_8y_d',
 'rpi_9y_d',
 'rpi_10y_d',
 'rpi_15y_d',
 'rpi_20y_d',
 'rpi_25y_d',
 'rpi_30y_d',
 'cpi_1m_d',
 'cpi_3m_d',
 'cpi_6m_d',
 'cpi_9m_d',
 'cpi_1y_d',
 'cpi_2y_d',
 'cpi_3y_d',
 'cpi_4y_d',
 'cpi_5y_d',
 'cpi_6y_d',
 'cpi_7y_d',
 'cpi_8y_d',
 'cpi_9y_d',
 'cpi_10y_d',
 'cpi_15y_d',
 'cpi_20y_d',
 'cpi_25y_d',
 'cpi_30y_d',
 'ois_1w_d',
 'ois_2w_d',
 'ois_3w_d',
 'ois_1m_d',
 'ois_2m_d',
 'ois_3m_d',
 'ois_4m_d',
 'ois_5m_d',

In [6]:
reg1 = regression(df, "10.0_n_d", "q1_ratio")
reg2 = regression(df, "10.0_n_d", "q1_ratio", "icr_d")
reg3 = regression(df, "10.0_n_d", "q1_ratio", "icrf_d")
reg4 = regression(df, "10.0_n_d", "q1_ratio", "ivwir_d")
reg5 = regression(df, "10.0_n_d", "q1_ratio", "ilr_sq_d")

Regression:
10.0_n_d ~ q1_ratio
Parameters:
const       0.049535
q1_ratio    0.215467
dtype: float64
R2: 0.1622865671861422
T-values:
const       2.046533
q1_ratio    4.152298
dtype: float64

Regression:
10.0_n_d ~ q1_ratio + icr_d + interaction
Parameters:
const          0.036391
q1_ratio       0.206897
icr_d          0.004409
interaction   -0.105163
dtype: float64
R2: 0.23951164513394496
T-values:
const          1.384574
q1_ratio       2.986160
icr_d          0.195641
interaction   -2.085454
dtype: float64

Regression:
10.0_n_d ~ q1_ratio + icrf_d + interaction
Parameters:
const          0.035064
q1_ratio       0.208100
icrf_d         0.001018
interaction   -0.110486
dtype: float64
R2: 0.2087001861865223
T-values:
const          1.285682
q1_ratio       2.930986
icrf_d         0.044434
interaction   -1.576909
dtype: float64

Regression:
10.0_n_d ~ q1_ratio + ivwir_d + interaction
Parameters:
const          0.036521
q1_ratio       0.212118
ivwir_d        0.003182
interaction   -0.10392

In [7]:
# show t values
print("T-values:")
print(reg1.tvalues)
print(reg2.tvalues)
print(reg3.tvalues)
print(reg4.tvalues)
print(reg5.tvalues)

T-values:
const       2.046533
q1_ratio    4.152298
dtype: float64
const          1.384574
q1_ratio       2.986160
icr_d          0.195641
interaction   -2.085454
dtype: float64
const          1.285682
q1_ratio       2.930986
icrf_d         0.044434
interaction   -1.576909
dtype: float64
const          1.337625
q1_ratio       2.985811
ivwir_d        0.138543
interaction   -1.456294
dtype: float64
const          1.292454
q1_ratio       3.136652
ilr_sq_d      -0.852958
interaction    2.293878
dtype: float64


In [8]:
df

,date,year,dow,note,date_n5,date_p1,icr_d,icrf_d,ivwir_d,ilr_sq_d,...,ois_12y_d,ois_15y_d,ois_20y_d,ois_25y_d,ois_30y_d,ois_40y_d,ois_50y_d,article_count,q1_ratio,q2_ratio
0,1980-03-26,1980,wed,spring_1,1980-03-19,1980-03-27,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,140,-0.589744,-0.661017
1,1981-03-10,1981,tue,spring_1,1981-03-03,1981-03-11,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,236,-0.369565,-0.776224
2,1982-03-09,1982,tue,spring_1,1982-03-02,1982-03-10,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,258,-0.375000,-0.671053
3,1982-11-08,1982,mon,autumn_1,1982-11-01,1982-11-09,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,68,-0.800000,-0.777778
4,1983-03-15,1983,tue,spring_1,1983-03-08,1983-03-16,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,210,-0.616438,-0.698113
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
86,2022-03-23,2022,wed,spring_1,2022-03-16,2022-03-24,0.137656,-1.041333,-1.017542,-0.180219,...,0.0281,0.0365,0.04700,0.04780,0.04900,0.0457,0.04140,26,0.500000,-0.882353
87,2022-09-23,2022,fri,mini_1,2022-09-15,2022-09-26,-1.611567,-0.991346,-0.852320,1.443618,...,1.0204,1.0015,0.96050,0.94050,0.92650,0.8987,0.87720,67,0.658537,-0.846154
88,2022-11-17,2022,thu,autumn_1,2022-11-10,2022-11-18,0.040904,-1.164408,-0.992952,-0.136375,...,-0.0508,-0.0580,-0.06190,-0.06080,-0.05980,-0.0656,-0.06910,33,0.058824,-1.000000
89,2023-03-15,2023,wed,spring_1,2023-03-08,2023-03-16,-1.597111,0.642808,0.655452,1.059468,...,-0.2450,-0.2400,-0.23000,-0.22100,-0.21200,-0.1974,-0.18930,24,-0.777778,-0.875000


In [9]:
df1 = df.copy()
df1['time_period'] = df1.apply(lambda x: f"{x['date_n5']}_to_{x['date_p1']}", axis=1)

In [10]:
df1 = df1[['time_period'] + df1.columns.tolist()[:-1]]

In [11]:
import re

name_change = {}

for col in df1.columns:
    if re.search(r"\_d$", col):
        new_name = re.sub(r"\_d$", "", col)
        name_change[col] = new_name
        
df1 = df1.rename(columns=name_change)

In [12]:
# na counts
pd.options.display.max_rows = None
pd.options.display.max_columns = None
print(df1.isna().sum().to_frame(name="NA Count"))

               NA Count
time_period           0
date                  0
year                  0
dow                   0
note                  0
date_n5               0
date_p1               0
icr                  37
icrf                 37
ivwir                37
ilr_sq               37
2.5_n                 0
5.0_n                 0
10.0_n                0
15.0_n                0
20.0_n                1
30.0_n               72
2.5_r                 8
5.0_r                 8
10.0_r                8
15.0_r                8
20.0_r               11
30.0_r               72
2.5_i                 8
5.0_i                 8
10.0_i                8
15.0_i                8
20.0_i               11
30.0_i               72
gbpusd_o             18
gbpusd_c             18
ftse_o               18
ftse_c               18
gbpeur_o             18
gbpeur_c             18
rpi_1m               80
rpi_3m               80
rpi_6m               80
rpi_9m               80
rpi_1y               46
rpi_2y          

In [13]:
to_drop_cols = [
    '30.0_n',
    '30.0_r',
    '30.0_i',
    'rpi_1m',
    'rpi_3m',
    'rpi_6m',
    'rpi_9m',
    'cpi_1m',
    'cpi_3m',
    'cpi_6m',
    'cpi_9m',
    'cpi_1y',
    'cpi_3y',
    'cpi_4y',
    'cpi_6y',
    'cpi_7y',
    'cpi_8y',
    'cpi_9y',
    'cpi_15y',
    'cpi_25y',
    'ois_2m',
    'ois_4m',
    'ois_5m',
    'ois_7m',
    'ois_8m',
    'ois_9m',
    'ois_10m',
    'ois_11m',
    'ois_18m',
    'ois_2y',
    'ois_3y',
    'ois_4y',
    'ois_6y',
    'ois_7y',
    'ois_8y',
    'ois_9y',
    'ois_12y',
    'ois_25y'
]

df1 = df1.drop(columns=to_drop_cols)

In [14]:
print(df1.isna().sum().to_frame(name="NA Count"))


               NA Count
time_period           0
date                  0
year                  0
dow                   0
note                  0
date_n5               0
date_p1               0
icr                  37
icrf                 37
ivwir                37
ilr_sq               37
2.5_n                 0
5.0_n                 0
10.0_n                0
15.0_n                0
20.0_n                1
2.5_r                 8
5.0_r                 8
10.0_r                8
15.0_r                8
20.0_r               11
2.5_i                 8
5.0_i                 8
10.0_i                8
15.0_i                8
20.0_i               11
gbpusd_o             18
gbpusd_c             18
ftse_o               18
ftse_c               18
gbpeur_o             18
gbpeur_c             18
rpi_1y               46
rpi_2y               46
rpi_3y               48
rpi_4y               48
rpi_5y               48
rpi_6y               48
rpi_7y               48
rpi_8y               48
rpi_9y          

In [22]:
# regress: 10.0_n ~ q1_ratio + ftse_o + gbpeur_o

reg = regression(df1, "10.0_n", "q1_ratio", "gbpeur_o")
print(reg.summary())

Regression:
10.0_n ~ q1_ratio + gbpeur_o + interaction
Parameters:
const           0.037878
q1_ratio        0.171182
gbpeur_o       -5.795478
interaction   -11.732627
dtype: float64
R2: 0.3987474969578718
T-values:
const          1.781779
q1_ratio       3.417454
gbpeur_o      -4.272696
interaction   -4.311794
dtype: float64

                            OLS Regression Results                            
Dep. Variable:                 10.0_n   R-squared:                       0.399
Model:                            OLS   Adj. R-squared:                  0.373
Method:                 Least Squares   F-statistic:                     15.25
Date:                Sun, 03 Nov 2024   Prob (F-statistic):           1.03e-07
Time:                        00:15:07   Log-Likelihood:                 45.267
No. Observations:                  73   AIC:                            -82.53
Df Residuals:                      69   BIC:                            -73.37
Df Model:                           3    